#ESE521 Final project submission
Team: Four finger punch

Author: Ziyi Yang, Miranda Wang


#How to run:

Check the raw ecog data: Please load the test data to variable 'truetest_data', change the name of the header in the dictionary file if needed.


Load the filter matrix in f_matrix.mat(f1,f2,f3)


Tested with the given leaderboard_ecog data with no error. Good for final submission

In [24]:
#Import all pkgs here
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from scipy.stats import pearsonr
from scipy import signal as sig
from scipy.signal import ellip,butter, filtfilt,sosfiltfilt,iirnotch
from scipy.io import loadmat
from scipy.signal import butter, sosfilt, sosfreqz
from sklearn import preprocessing
from numpy.linalg import inv

from google.colab import drive
drive.mount('/content/drive')

#Import raw data
truetest_data = loadmat('/content/drive/MyDrive/BE521/FinalProject/raw_training_data.mat')
#truetest_data = loadmat('truetest_data.mat')

#loading f matrix
mat=loadmat("/content/f_matrix.mat")
f1=mat["f_matrix"][0][0]
f2=mat["f_matrix"][1][0]
f3=mat["f_matrix"][2][0]

#split true test data
# ecog_1 = np.array(truetest_data['truetest_ecog'][0][0])
# ecog_2 = np.array(truetest_data['truetest_ecog'][1][0])
# ecog_3 = np.array(truetest_data['truetest_ecog'][2][0])

ecog_1 = np.array(truetest_data['train_ecog'][0][0])
ecog_2 = np.array(truetest_data['train_ecog'][1][0])
ecog_3 = np.array(truetest_data['train_ecog'][2][0])


#add 37ms delay to the true test data
ecog_1[37:]=ecog_1[0:len(ecog_1)-37]
ecog_2[37:]=ecog_2[0:len(ecog_1)-37]
ecog_3[37:]=ecog_3[0:len(ecog_1)-37]



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
# #split training data glove data###Test for submission
# dg_1 = np.array(truetest_data['train_dg'][0][0])
# dg_2 = np.array(truetest_data['train_dg'][1][0])
# dg_3 = np.array(truetest_data['train_dg'][2][0])
# #calculating correlation
# def correlation_coefficient(T1, T2):
#     numerator = np.mean((T1 - np.mean(T1)) * (T2 - np.mean(T1)))
#     denominator = np.std(T1) * np.std(T2)
#     if denominator == 0:
#         return 0
#     else:
#         result = numerator / denominator
#         return result

In [20]:
#all functions here
#extract good channels
ind = np.array([np.arange(0, 5), np.array([9]), np.arange(14, 28), np.arange(33, 47)])
ind1 = []
for arr in ind:
    ind1.extend(arr)
aa = np.array([6, 7, 8,9,10,11,12])
bb = np.arange(14,29)
cc = np.array([30,33,34,35,37,42,43,45])
ind = [aa, bb, cc]
ind2 = []
for arr in ind:
    ind2.extend(arr)
ind = [np.arange(6,29), np.array([33,34,40,43,45,47,48,50,53,54,56,60])]
ind3 = []
for arr in ind:
    ind3.extend(arr)

#function to delete channel
def del_ch(arr,ch):
  new_arr=[]
  for i in range(len(arr)):
    temp=np.delete(arr[i],ch)
    new_arr.append(temp)
  new_arr = np.array(new_arr)
  return new_arr

#function to filter data
def car(data):
  for i in range(np.shape(data)[0]):
    mean=np.mean(data[i,:])
    for j in range(np.shape(data)[1]):
      data[i,j]-=mean

def filter_data(data, fs=1000,bandpass=[1,490],order=2):
  data_t = data.transpose()
  result = []
  nyq = fs/2
  sos = butter(order,bandpass,btype='bandpass',output='sos',fs=fs)
  for i in data_t:
    temp= sosfiltfilt(sos,i)
    result.append(temp)
  
  result = np.array(result)
  return result.transpose()

#Feature extraction functions#

#time domain to frequency domain
#calculate bandpower
from scipy.signal import welch
def bandpower(x, fs=1000,band=[5,15],time=0.1):
    # f, Pxx = scipy.signal.periodogram(x, fs=fs)
    #f, Pxx = welch(x, fs, nperseg=fs*time)
    fmin=band[0]
    fmax=band[1]
    freq_data = np.fft.fft(x)
    frequency = np.linspace (0.0, 500, int(100/2))
    y = 2/100 * np.abs (freq_data [0:int(100/2)])
    ind_min = np.argmax(frequency > fmin) - 1
    ind_max = np.argmax(frequency > fmax) - 1
    return np.mean(y[ind_min:ind_max])

#average voltage feature
def ave_voltage(data):
  sum=np.sum(data)
  ave=sum/len(data)
  return ave

#line length feature
def LL(data):
  result = np.sum(np.absolute(np.ediff1d(data)))
  return result

def Area(data):
  result = np.sum(np.absolute(data))
  return result

def E(data):
  temp=[]
  for i in data:
    temp.append(i*i)
  return np.sum(temp)


def get_features_1(filtered_window, fs=1000):
  cn=filtered_window.shape[1]
  result=[]
  for i in range(cn):
    temp=filtered_window[:,i]
    ll=LL(temp)
    result.append(ll)
  a=np.divide((result-np.mean(result)),np.std(result))
  return a

def get_features_2(filtered_window, fs=1000):
  cn=filtered_window.shape[1]
  result=[]
  for i in range(cn):
    temp=filtered_window[:,i]
    e=E(temp)
    result.append([e])
  return result 

#getting feats for each window
def NumWins(x, fs, winLen, winDisp):
  return (len(x) - winLen*fs + winDisp*fs) // (winDisp * fs)

def get_windowed_feats_1(raw_ecog, fs=1000, window_length=100, window_overlap=50):
  numWins = NumWins(raw_ecog,1000,0.1,0.05)
  i = 0
  pos = 0
  curr_winFeat = []
  result = []
  while(i<numWins):
    curr_winFeat = get_features_1(raw_ecog[pos:(pos+window_length)])
    result.append(curr_winFeat)
    pos +=window_overlap
    i += 1
  result = np.array(result)
  return result

def get_windowed_feats_2(raw_ecog, fs=1000, window_length=100, window_overlap=50):
  numWins = NumWins(raw_ecog,1000,0.1,0.05)
  i = 0
  pos = 0
  curr_winFeat = []
  result = []
  while(i<numWins):
    curr_winFeat = get_features_2(raw_ecog[pos:(pos+window_length)])
    result.append(curr_winFeat)
    pos +=window_overlap
    i += 1
  result = np.array(result)
  return result

  #start at 150s orginal data
#downsample function
def get_ave(filtered_window, fs=1000):
  cn=filtered_window.shape[1]
  result=[]
  for i in range(cn):
    temp=filtered_window[:,i]
    ave=ave_voltage(temp)
    result.append(ave)
  result = np.array(result)
  return result 
def downsample_data(raw_ecog, fs=1000, window_length=100, window_overlap=50):
  numWins = NumWins(raw_ecog,1000,0.1,0.05)
  i = 0
  pos = 0
  curr_winFeat = []
  result = []
  while(i<numWins):
    curr_winFeat = get_ave(raw_ecog[pos:(pos+window_length)])
    result.append(curr_winFeat)
    pos +=window_overlap
    i += 1
  result = np.array(result)
  return result

def upsample_data(data,sample,up_sample):
  from scipy.interpolate import CubicSpline
  x=np.linspace(0,up_sample-100,sample)
  cs = CubicSpline(x, data)
  xx=np.arange(up_sample)
  arr_upsampled=np.array(cs(xx))
  return arr_upsampled

def us(data):
  result=[]
  for i in data[:len(data)-50]:
    for j in range(50):
      result.append(i)
  for i in data[len(data)-50:]:
    for j in range(51):
      result.append(i)
  return np.array(result)

#funtion to create R matrix
def R_matrix(features,N_wind):
  R = np.zeros((np.shape(features)[0],(np.shape(features)[1]*N_wind)+1))
  R[:,0]=1
  aa=[features[0,:],features[1,:],features[2,:],features[3,:],features[4,:],features[5,:],features[6,:],features[7,:],features[8,:],features[0,:]]
  aa=np.reshape(aa,(np.shape(aa)[0]*np.shape(aa)[1],))
  R[0,1:]=aa
  aa=[features[1,:],features[2,:],features[3,:],features[4,:],features[5,:],features[6,:],features[7,:],features[8,:],features[0,:],features[1,:]]
  aa=np.reshape(aa,(np.shape(aa)[0]*np.shape(aa)[1],))
  R[1,1:]=aa
  aa=[features[2,:],features[3,:],features[4,:],features[5,:],features[6,:],features[7,:],features[8,:],features[0,:],features[1,:],features[2,:]]
  aa=np.reshape(aa,(np.shape(aa)[0]*np.shape(aa)[1],))
  R[2,1:]=aa
  aa=[features[3,:],features[4,:],features[5,:],features[6,:],features[7,:],features[8,:],features[0,:],features[1,:],features[2,:],features[3,:]]
  aa=np.reshape(aa,(np.shape(aa)[0]*np.shape(aa)[1],))
  R[3,1:]=aa
  aa=[features[4,:],features[5,:],features[6,:],features[7,:],features[8,:],features[0,:],features[1,:],features[2,:],features[3,:],features[4,:]]
  aa=np.reshape(aa,(np.shape(aa)[0]*np.shape(aa)[1],))
  R[4,1:]=aa
  aa=[features[5,:],features[6,:],features[7,:],features[8,:],features[0,:],features[1,:],features[2,:],features[3,:],features[4,:],features[5,:]]
  aa=np.reshape(aa,(np.shape(aa)[0]*np.shape(aa)[1],))
  R[5,1:]=aa
  aa=[features[6,:],features[7,:],features[8,:],features[0,:],features[1,:],features[2,:],features[3,:],features[4,:],features[5,:],features[6,:]]
  aa=np.reshape(aa,(np.shape(aa)[0]*np.shape(aa)[1],))
  R[6,1:]=aa
  aa=[features[7,:],features[8,:],features[0,:],features[1,:],features[2,:],features[3,:],features[4,:],features[5,:],features[6,:],features[7,:]]
  aa=np.reshape(aa,(np.shape(aa)[0]*np.shape(aa)[1],))
  R[7,1:]=aa
  aa=[features[8,:],features[0,:],features[1,:],features[2,:],features[3,:],features[4,:],features[5,:],features[6,:],features[7,:],features[8,:]]
  aa=np.reshape(aa,(np.shape(aa)[0]*np.shape(aa)[1],))
  R[8,1:]=aa
  arr = np.arange(9,np.shape(features)[0],1)
  for i in arr:
    i-=1
    aa=[features[i-8,:],features[i-7,:],features[i-6,:],features[i-5,:],features[i-4,:],features[i-3,:],features[i-2,:],features[i-1,:],features[i,:],features[i+1,:]]
    aa=np.reshape(aa,(np.shape(aa)[0]*np.shape(aa)[1],))
    R[i+1,1:]=aa
  return R


#reshape ecog winfeats
def reshape(ecog_winfeats):
  result=np.reshape(ecog_winfeats,(len(ecog_winfeats),np.shape(ecog_winfeats)[2]*np.shape(ecog_winfeats)[1]))
  return np.array(result)

def Y_bar(R_matrix,f_matrix):
  result=np.matmul(R_matrix,f_matrix)
  return np.array(result)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
#removing outliers ch55 in subject1/ch21 ch38 in subject2
ecog_1 = del_ch(ecog_1,54)
ecog_2 = del_ch(ecog_2,20)
ecog_2 = del_ch(ecog_2,36)
#select good channels
ecog_1 = ecog_1[:,ind1]
ecog_2 = ecog_2[:,ind2]
ecog_3 = ecog_3[:,ind3]

#filtering ecog data
ecog_1_filtered = filter_data(ecog_1,bandpass=[105,490])
ecog_2_filtered = filter_data(ecog_2,bandpass=[60,490])
ecog_3_filtered = filter_data(ecog_3,bandpass=[60,490])
car(ecog_1_filtered)
car(ecog_2_filtered)
car(ecog_3_filtered)
#get windowed features
ecog_1_winFeats=get_windowed_feats_1(ecog_1_filtered)
ecog_2_winFeats=get_windowed_feats_2(ecog_2_filtered)
ecog_3_winFeats=get_windowed_feats_2(ecog_3_filtered)

#calculating R matrix
ecog_1_rs = ecog_1_winFeats
ecog_2_rs = reshape(ecog_2_winFeats)
ecog_3_rs = reshape(ecog_3_winFeats)

R_1_matrix = R_matrix(ecog_1_rs,10)
R_2_matrix = R_matrix(ecog_2_rs,10)
R_3_matrix = R_matrix(ecog_3_rs,10)

#Making prediction
Y1b = Y_bar(R_1_matrix,f1)
Y1b_us = us(Y1b)

Y2b = Y_bar(R_2_matrix,f2)
Y2b_us = us(Y2b)

Y3b = Y_bar(R_3_matrix,f3)
Y3b_us = us(Y3b)

# print(correlation_coefficient(Y1b_us,dg_1))
# print(correlation_coefficient(Y2b_us,dg_2))
# print(correlation_coefficient(Y3b_us,dg_3))

0.6029984279038292
0.5295352610248476
0.724098395052174


In [ ]:
#saving predicted_dg.mat

import numpy as np
from scipy.io import savemat
#create an example submission array
predictions = np.zeros((3,1), dtype=object)
predictions[0,0] = Y1b_us
predictions[1,0] = Y2b_us
predictions[2,0] = Y3b_us
#save the array using the right format
savemat('predictions.mat', {'predicted_dg':predictions})